In [1]:
import pandas as pd
import pandas as pd
import nltk
from nltk import ngrams
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
##imports
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import os
from tqdm.asyncio import tqdm as async_tqdm
import nest_asyncio
from openai import AsyncOpenAI
from typing import Optional
from tqdm.asyncio import tqdm as async_tqdm
from spacy.lang.ja import Japanese
from spacy.lang.ru import Russian
from spacy.lang.fi import Finnish
import nltk
#download the knbc corpus
nltk.download('knbc')
nltk.download('stopwords')
from nltk.corpus import knbc
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

current_dir = os.getcwd()
if current_dir.endswith("code"):
    os.chdir("..")
else:
    print("current dir", current_dir)

nest_asyncio.apply()

[nltk_data] Downloading package knbc to /Users/HansPeter/nltk_data...
[nltk_data]   Package knbc is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/HansPeter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/HansPeter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/HansPeter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
ds_train = pd.read_parquet("dataset/train_df.parquet")
ds_val = pd.read_parquet("dataset/val_df.parquet")

In [3]:

# Download necessary NLTK data (if not already downloaded)
nltk.download('punkt')

# Assuming you have your text data in a pandas DataFrame
# For example: df = pd.read_csv('your_data.csv')

# Tokenize your text and create n-grams
n = 2  # for bigrams, change as needed
text = df['text_column'].tolist()
tokenized_text = [nltk.word_tokenize(sent) for sent in text]

# Prepare the data for training
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

# Create and train the model
model = MLE(n)  # MLE = Maximum Likelihood Estimation
model.fit(train_data, padded_sents)

# Now you can use the model for various tasks, e.g., calculating probabilities


[nltk_data] Downloading package punkt to /Users/HansPeter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NameError: name 'df' is not defined